In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import *

In [2]:
# DAYS not to be considered
relevant_dates =  [
    "2010-03-08", "2010-06-08", "2010-09-08", "2010-12-08",
    "2011-03-08", "2011-06-08", "2011-09-08", "2011-12-08",
    "2012-03-08", "2012-06-07", "2012-09-06", "2012-12-06",
    "2013-03-07", "2013-06-06", "2013-09-06", "2013-12-06",
    "2014-03-06", "2014-06-06", "2014-09-08", "2014-12-08",
    "2015-03-06", "2015-06-08", "2015-09-08", "2015-12-08",
    "2016-03-08", "2016-06-08", "2016-09-08", "2016-12-08",
    "2017-03-08", "2017-06-08", "2017-09-07", "2017-12-07",
    "2018-03-08", "2018-06-07", "2018-09-06", "2018-12-06",
    "2019-03-07", "2019-06-06", "2019-09-06", "2019-12-06",
    "2020-03-06", "2020-06-08", "2020-09-08", "2020-12-08",
    "2021-03-08", "2021-06-08", "2021-09-08", "2021-12-08",
    "2022-03-08", "2022-06-08", "2022-09-08", "2022-12-08",
    "2023-03-08", "2023-06-08", "2023-09-07", "2023-12-07",
]

anomalous_days = ["2014-05-26", "2015-02-17", "2015-07-20", "2016-07-14", "2018-03-16", "2020-01-22", "2022-08-29", "2023-07-24"]

## IKA Data

In [61]:
# Load volume data for all years into a single DataFrame, C1 data
file_TRADES_names = [
    f'../../data/IKA_trades_{year}_c1_partial.csv' if year == 2023
    else f'../../data/IKA_trades_{year}_c1.csv'
    for year in range(2010, 2024)
]

all_IKA_data = pd.concat([pd.read_csv(file) for file in file_TRADES_names])
all_IKA_data['Date-Time'] = pd.to_datetime(all_IKA_data['Date-Time'])
all_IKA_data.set_index('Date-Time',inplace=True)
all_IKA_data.drop('Volume', axis=1,inplace=True)

for a in anomalous_days:
    all_IKA_data = all_IKA_data[all_IKA_data.index.date != datetime.strptime(a, "%Y-%m-%d").date()]
    
for rel_date in relevant_dates:
    all_IKA_data = all_IKA_data[all_IKA_data.index.date != datetime.strptime(rel_date, "%Y-%m-%d").date()]
    

In [62]:
all_IKA_data.index = pd.to_datetime(all_IKA_data.index).to_period('T')

In [63]:
all_IKA_data

,Price
Date-Time,
2010-01-04 08:12,114.71000
2010-01-04 08:17,114.71000
2010-01-04 08:22,114.68000
2010-01-04 08:23,114.66000
2010-01-04 08:48,114.60000
...,...
2023-08-31 18:55,115.84536
2023-08-31 18:56,115.85667
2023-08-31 18:57,115.86000


#### Delta price

In [64]:
deltaIntervals = [x for x in range(20)]

for interval in deltaIntervals:
    if (interval==0):
        all_IKA_data[f'DeltaPriceT{interval}'] = (all_IKA_data['Price']-all_IKA_data['Price'].shift(1))/all_IKA_data['Price'].shift(1)
    else:
        all_IKA_data[f'DeltaPriceT{interval}'] = all_IKA_data[f'DeltaPriceT{interval-1}'].shift(1)

all_IKA_data

,Price,DeltaPriceT0,DeltaPriceT1,DeltaPriceT2,DeltaPriceT3,DeltaPriceT4,DeltaPriceT5,DeltaPriceT6,DeltaPriceT7,DeltaPriceT8,...,DeltaPriceT10,DeltaPriceT11,DeltaPriceT12,DeltaPriceT13,DeltaPriceT14,DeltaPriceT15,DeltaPriceT16,DeltaPriceT17,DeltaPriceT18,DeltaPriceT19
Date-Time,,,,,,,,,,,,,,,,,,,,,
2010-01-04 08:12,114.71000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:17,114.71000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:22,114.68000,-0.000262,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:23,114.66000,-0.000174,-0.000262,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:48,114.60000,-0.000523,-0.000174,-0.000262,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31 18:55,115.84536,-0.000040,-0.000008,-0.000050,0.000070,0.000071,-0.000127,-0.000126,-0.000116,-0.000018,...,-0.000082,-0.000027,0.000047,-0.000118,-0.000108,-0.000046,-0.000041,-0.000054,-0.000032,0.000000
2023-08-31 18:56,115.85667,0.000098,-0.000040,-0.000008,-0.000050,0.000070,0.000071,-0.000127,-0.000126,-0.000116,...,0.000247,-0.000082,-0.000027,0.000047,-0.000118,-0.000108,-0.000046,-0.000041,-0.000054,-0.000032
2023-08-31 18:57,115.86000,0.000029,0.000098,-0.000040,-0.000008,-0.000050,0.000070,0.000071,-0.000127,-0.000126,...,-0.000018,0.000247,-0.000082,-0.000027,0.000047,-0.000118,-0.000108,-0.000046,-0.000041,-0.000054


#### Distance from Roll (in days)

In [65]:
all_IKA_data_dateTime = all_IKA_data.index.to_timestamp().date

In [66]:
relevant_dates_dt = pd.to_datetime(relevant_dates).date
distance_fromRoll = []
currentRollDateIndex = 0
for i in range(len(all_IKA_data_dateTime)):
    if (all_IKA_data_dateTime[i] > relevant_dates_dt[currentRollDateIndex]):
        currentRollDateIndex += 1
    currentDistance = (relevant_dates_dt[currentRollDateIndex] - all_IKA_data_dateTime[i]).days
    distance_fromRoll.append(currentDistance)
    
all_IKA_data['DistanceFromRoll'] = distance_fromRoll

In [67]:
all_IKA_data.to_csv('all_IKA_data.csv',index=True)

## LOB Data

In [3]:
file_LOB_names = [
    f'../../data/IKA_LOB_{year}_c1_partial.csv' if year == 2023
    else f'../../data/IKA_LOB_{year}_c1.csv'
    for year in range(2010, 2024)
]
columns_to_keep = ["Date-Time","L1-BidPrice","L1-BidSize","L1-AskPrice","L1-AskSize"]

all_LOB_data = pd.concat([pd.read_csv(file) for file in file_LOB_names])
all_LOB_data['Date-Time'] = pd.to_datetime(all_LOB_data['Date-Time'])
all_LOB_data.set_index('Date-Time',inplace=True)
all_LOB_data = all_LOB_data[all_LOB_data.columns.intersection(columns_to_keep)]

#Removing days not to be considered
for a in anomalous_days:
    all_LOB_data = all_LOB_data[all_LOB_data.index.date != datetime.strptime(a, "%Y-%m-%d").date()]
    
for rel_date in relevant_dates:
    all_LOB_data = all_LOB_data[all_LOB_data.index.date != datetime.strptime(rel_date, "%Y-%m-%d").date()]

all_LOB_data

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize
Date-Time,,,,
2010-01-04 08:01:57.425660,114.41,10,114.61,10
2010-01-04 08:02:56.396773,114.44,1,114.60,10
2010-01-04 08:03:56.618427,114.49,7,114.63,10
2010-01-04 08:04:57.213687,114.55,1,114.62,10
2010-01-04 08:05:50.611971,114.55,1,114.65,10
...,...,...,...,...
2023-08-31 18:56:58.958861,115.85,4,115.86,11
2023-08-31 18:57:58.729024,115.85,5,115.86,16
2023-08-31 18:58:53.709708,115.85,2,115.86,27


In [4]:
all_LOB_data.index = pd.to_datetime(all_LOB_data.index).to_period('T')
all_LOB_data['Spread'] = all_LOB_data['L1-AskPrice'] - all_LOB_data['L1-BidPrice']
all_LOB_data['midPrice'] = (all_LOB_data['L1-AskPrice'] + all_LOB_data['L1-BidPrice']) / 2
all_LOB_data

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,Spread,midPrice
Date-Time,,,,,,
2010-01-04 08:01,114.41,10,114.61,10,0.20,114.510
2010-01-04 08:02,114.44,1,114.60,10,0.16,114.520
2010-01-04 08:03,114.49,7,114.63,10,0.14,114.560
2010-01-04 08:04,114.55,1,114.62,10,0.07,114.585
2010-01-04 08:05,114.55,1,114.65,10,0.10,114.600
...,...,...,...,...,...,...
2023-08-31 18:56,115.85,4,115.86,11,0.01,115.855
2023-08-31 18:57,115.85,5,115.86,16,0.01,115.855
2023-08-31 18:58,115.85,2,115.86,27,0.01,115.855


#### Delta Mid Price computation

In [10]:
one_hour = pd.Timedelta(hours=1)
maximum_delta_time = pd.Timedelta(minutes=5)

def find_nearest_timestamp(current_time):
    delta = pd.Timedelta(0)
    while delta <= maximum_delta_time:
        if current_time - delta in all_LOB_data.index:
            return current_time - delta
        elif current_time + delta in all_LOB_data.index:
            return current_time + delta
        else:
            delta += pd.Timedelta(minutes=1)

    return None

def calculate_delta_mid_price(row):
    index_one_hour_before = find_nearest_timestamp(row.name - pd.Timedelta(hours=1))
    if index_one_hour_before is not None:
        row_one_hour_before_mid_price = all_LOB_data.loc[index_one_hour_before, 'midPrice']
        delta_mid_price = (row['midPrice'] - row_one_hour_before_mid_price) / row_one_hour_before_mid_price
        return delta_mid_price
    else:
        return None

    
all_LOB_data['DeltaMidPrice'] = all_LOB_data.apply(calculate_delta_mid_price, axis=1)

In [11]:
all_LOB_data

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,Spread,midPrice,DeltaMidPrice
Date-Time,,,,,,,
2010-01-04 08:01,114.41,10,114.61,10,0.20,114.510,NaN
2010-01-04 08:02,114.44,1,114.60,10,0.16,114.520,NaN
2010-01-04 08:03,114.49,7,114.63,10,0.14,114.560,NaN
2010-01-04 08:04,114.55,1,114.62,10,0.07,114.585,NaN
2010-01-04 08:05,114.55,1,114.65,10,0.10,114.600,NaN
...,...,...,...,...,...,...,...
2023-08-31 18:56,115.85,4,115.86,11,0.01,115.855,0.000086
2023-08-31 18:57,115.85,5,115.86,16,0.01,115.855,0.000086
2023-08-31 18:58,115.85,2,115.86,27,0.01,115.855,0.000000


In [70]:
imbalance = []
for i in range(len(all_LOB_data)):
    if (i==0): 
        imbalance.append(0)
    else:
        currentRow = all_LOB_data.iloc[i]
        previousRow = all_LOB_data.iloc[i-1]
        
        imbalanceVariation = 0
        demandVariation = 0
        supplyVariation = 0
        
        if (currentRow['L1-BidPrice'] > previousRow['L1-BidPrice']):
            demandVariation = currentRow['L1-BidSize']
        elif (currentRow['L1-BidPrice'] < previousRow['L1-BidPrice']):
            demandVariation = -previousRow['L1-BidSize']
        else:
            demandVariation = currentRow['L1-BidSize'] - previousRow['L1-BidSize']
            
        if (currentRow['L1-AskPrice'] < previousRow['L1-AskPrice']):
            supplyVariation = -currentRow['L1-AskSize']
        elif (currentRow['L1-AskPrice'] > previousRow['L1-AskPrice']):
            supplyVariation = previousRow['L1-AskSize']
        else:
            supplyVariation = -currentRow['L1-AskSize'] + previousRow['L1-AskSize']
            
        imbalanceVariation = demandVariation + supplyVariation
        imbalance.append(imbalanceVariation)

In [71]:
all_LOB_data['imbalanceEvent'] = imbalance

In [72]:
all_LOB_data.to_csv('all_LOB_data.csv',index=True)

## Auction Data

#### Merging the two DataFrames

In [92]:
all_LOB_data = pd.read_csv('all_LOB_data.csv',index_col=0)
all_IKA_data = pd.read_csv('all_IKA_data.csv',index_col=0)

In [93]:
all_LOB_data

,L1-BidPrice,L1-BidSize,L1-AskPrice,L1-AskSize,Spread,imbalanceEvent
Date-Time,,,,,,
2010-01-04 08:01,114.41,10,114.61,10,0.20,0.0
2010-01-04 08:02,114.44,1,114.60,10,0.16,-9.0
2010-01-04 08:03,114.49,7,114.63,10,0.14,17.0
2010-01-04 08:04,114.55,1,114.62,10,0.07,-9.0
2010-01-04 08:05,114.55,1,114.65,10,0.10,10.0
...,...,...,...,...,...,...
2023-08-31 18:56,115.85,4,115.86,11,0.01,-1.0
2023-08-31 18:57,115.85,5,115.86,16,0.01,-4.0
2023-08-31 18:58,115.85,2,115.86,27,0.01,-14.0


In [94]:
all_LOB_data.drop(columns=['L1-BidPrice','L1-BidSize','L1-AskPrice','L1-AskSize'],inplace=True)

In [95]:
all_IKA_data

,Price,DeltaPriceT0,DeltaPriceT1,DeltaPriceT2,DeltaPriceT3,DeltaPriceT4,DeltaPriceT5,DeltaPriceT6,DeltaPriceT7,DeltaPriceT8,...,DeltaPriceT11,DeltaPriceT12,DeltaPriceT13,DeltaPriceT14,DeltaPriceT15,DeltaPriceT16,DeltaPriceT17,DeltaPriceT18,DeltaPriceT19,DistanceFromRoll
Date-Time,,,,,,,,,,,,,,,,,,,,,
2010-01-04 08:12,114.71000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
2010-01-04 08:17,114.71000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
2010-01-04 08:22,114.68000,-0.000262,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
2010-01-04 08:23,114.66000,-0.000174,-0.000262,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
2010-01-04 08:48,114.60000,-0.000523,-0.000174,-0.000262,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31 18:55,115.84536,-0.000040,-0.000008,-0.000050,0.000070,0.000071,-0.000127,-0.000126,-0.000116,-0.000018,...,-0.000027,0.000047,-0.000118,-0.000108,-0.000046,-0.000041,-0.000054,-0.000032,0.000000,7
2023-08-31 18:56,115.85667,0.000098,-0.000040,-0.000008,-0.000050,0.000070,0.000071,-0.000127,-0.000126,-0.000116,...,-0.000082,-0.000027,0.000047,-0.000118,-0.000108,-0.000046,-0.000041,-0.000054,-0.000032,7
2023-08-31 18:57,115.86000,0.000029,0.000098,-0.000040,-0.000008,-0.000050,0.000070,0.000071,-0.000127,-0.000126,...,0.000247,-0.000082,-0.000027,0.000047,-0.000118,-0.000108,-0.000046,-0.000041,-0.000054,7


In [96]:
all_IKA_data.drop(columns='Price',inplace=True)

In [97]:
merged_DF = pd.merge(all_LOB_data,all_IKA_data , left_index=True, right_index=True, how='inner')

#### Filtering data between 9:00 and 17:00

In [98]:
# Considering only values between 9:00 and 17:00
new_ind = pd.to_datetime(merged_DF.index).time
start_time = pd.to_datetime('09:00').time()
end_time = pd.to_datetime('17:00').time()

merged_DF_9_17 = merged_DF[(new_ind >= start_time) & (new_ind <= end_time)]

#### Adding distance from BTP Auctions

In [99]:
auctionDF = pd.read_csv('../../data/IKA_auctions.csv')
merged_DF_9_17.index = pd.to_datetime(merged_DF_9_17.index).to_period('T')
merged_DF_9_17_dateTime = merged_DF_9_17.index.to_timestamp().date

In [100]:
auctionDF_BTP = auctionDF[auctionDF['type']=="BTP"]
auctionDF_BTP_10Y = auctionDF_BTP[(auctionDF_BTP['maturity']>9) & (auctionDF_BTP['maturity']<11)]
auctionDF_BTP_10Y['date'] = pd.to_datetime(auctionDF_BTP_10Y['date'])
auctionDF_BTP_10Y = auctionDF_BTP_10Y[auctionDF_BTP_10Y['date'] > pd.to_datetime('2010-01-01')]

C:\Users\alber\AppData\Local\Temp\ipykernel_26728\44353428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auctionDF_BTP_10Y['date'] = pd.to_datetime(auctionDF_BTP_10Y['date'])


In [119]:
# Calculating distance from closest 10Y BTP auction
distanceFromAuction = []
auctionDates_BTP_10Y = auctionDF_BTP_10Y['date']
currentAuctionDate_index = 0

for i in range(len(merged_DF_9_17_dateTime)):
    if (merged_DF_9_17_dateTime[i] > auctionDates_BTP_10Y.iloc[currentAuctionDate_index].date()): 
        currentAuctionDate_index+=1
    currentDistanceFromAuction = (auctionDates_BTP_10Y.iloc[currentAuctionDate_index].date() - merged_DF_9_17_dateTime[i]).days
    distanceFromAuction.append(currentDistanceFromAuction)
    
merged_DF_9_17['distanceFromAuction'] = distanceFromAuction

C:\Users\alber\AppData\Local\Temp\ipykernel_26728\2849529798.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_DF_9_17['distanceFromAuction'] = distanceFromAuction


#### Adding Time feature

In [120]:
merged_DF_9_17['Time'] = merged_DF_9_17.index
merged_DF_9_17.reset_index(drop=True,inplace=True)
merged_DF_9_17

C:\Users\alber\AppData\Local\Temp\ipykernel_26728\854139359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_DF_9_17['Time'] = merged_DF_9_17.index


,Spread,imbalanceEvent,DeltaPriceT0,DeltaPriceT1,DeltaPriceT2,DeltaPriceT3,DeltaPriceT4,DeltaPriceT5,DeltaPriceT6,DeltaPriceT7,...,DeltaPriceT13,DeltaPriceT14,DeltaPriceT15,DeltaPriceT16,DeltaPriceT17,DeltaPriceT18,DeltaPriceT19,DistanceFromRoll,distanceFromAuction,Time
0,0.04,-19.0,0.000960,-0.000523,-0.000174,-0.000262,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,24,2010-01-04 09:31
1,0.06,53.0,0.000349,0.000960,-0.000523,-0.000174,-0.000262,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,24,2010-01-04 09:40
2,0.05,19.0,0.000087,0.000349,0.000960,-0.000523,-0.000174,-0.000262,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,24,2010-01-04 09:42
3,0.03,4.0,0.000174,0.000087,0.000349,0.000960,-0.000523,-0.000174,-0.000262,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,24,2010-01-04 10:05
4,0.01,25.0,0.000348,0.000174,0.000087,0.000349,0.000960,-0.000523,-0.000174,-0.000262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,24,2010-01-04 10:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483393,0.01,111.0,0.000185,-0.000008,0.000059,-0.000111,-0.000081,-0.000043,0.000197,0.000275,...,-0.000040,0.000068,-0.000013,-0.000145,0.000008,-0.000166,-0.000126,7,28,2023-08-31 16:56
1483394,0.01,-158.0,-0.000044,0.000185,-0.000008,0.000059,-0.000111,-0.000081,-0.000043,0.000197,...,0.000020,-0.000040,0.000068,-0.000013,-0.000145,0.000008,-0.000166,7,28,2023-08-31 16:57
1483395,0.02,-105.0,-0.000220,-0.000044,0.000185,-0.000008,0.000059,-0.000111,-0.000081,-0.000043,...,-0.000132,0.000020,-0.000040,0.000068,-0.000013,-0.000145,0.000008,7,28,2023-08-31 16:58
1483396,0.01,40.0,0.000005,-0.000220,-0.000044,0.000185,-0.000008,0.000059,-0.000111,-0.000081,...,0.000081,-0.000132,0.000020,-0.000040,0.000068,-0.000013,-0.000145,7,28,2023-08-31 16:59


In [121]:
merged_DF_9_17.to_csv('merged_DF.csv')